In [ ]:
import numpy as np
from scipy import io
from sklearn.mixture import GaussianMixture
import os

import holoviews as hv; hv.extension('bokeh', 'plotly', logo=False)
from holoviews.operation import contours as hv_contours
import panel as pn;     pn.extension()
import plotly.graph_objs as go

In [ ]:
dogdata_w_mat = io.loadmat(os.path.join('..','DATA','dogData_w.mat'))
catdata_w_mat = io.loadmat(os.path.join('..','DATA','catData_w.mat'))

dog_wave      = dogdata_w_mat['dog_wave']
cat_wave      = catdata_w_mat['cat_wave']

CD            = np.concatenate((dog_wave,cat_wave),axis=1)

u,s,vT        = np.linalg.svd(CD-np.mean(CD),full_matrices=0)
v             = vT.T

In [ ]:
dogcat  = v[:,(1,3)]
GMModel = GaussianMixture(n_components=2).fit(dogcat)
AIC     = GMModel.aic(dogcat)

x       = np.linspace(-0.15, 0.25)
y       = np.linspace(-0.25, 0.2)
X, Y    = np.meshgrid(x, y)
XX      = np.array([X.ravel(), Y.ravel()]).T
Z       = np.exp(GMModel.score_samples(XX))
Z       = Z.reshape(X.shape)


h_pts   = hv.Scatter((v[:80,1],v[:80,3])).opts(color='red', size=5) *\
          hv.Scatter((v[80:,1],v[80:,3])).opts(color='darkgreen', size=5)

(hv_contours(hv.Image((x,y,Z)))*h_pts).opts(show_legend=False, width=500, height=500)

In [ ]:
hv.output( backend='plotly', size=100 )

def plot_wireframe(xx, yy, z, color='#0066FF', linewidth=1, title=''):
    line_marker = dict(color=color, width=linewidth)
    lines = []
    for i, j, k in zip(xx, yy, z):
        lines.append(go.Scatter3d(x=i, y=j, z=k, mode='lines', line=line_marker))
        
    layout = go.Layout(showlegend=False, title=dict( text=title))
    return go.Figure(data=lines, layout=layout)

camera = dict(
    up     = dict(x=0, y=0, z=1),
    center = dict(x=0, y=0, z=0),
    eye    = dict(x=3, y=-2, z=1.25)
)


fig = plot_wireframe(X, Y, Z)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0), scene_camera=camera )

pn.pane.Plotly(fig, width=200)

In [ ]:
## AIC Scores
AIC = np.zeros(4)
GMModels = [GaussianMixture(k+1,covariance_type='diag',max_iter=500).fit(dogcat)
          for k in range(4)]

figs = []
for j in range(4):
    Z      = np.exp(GMModels[j].score_samples(XX))
    Z      = Z.reshape(X.shape)
    AIC[j] = GMModels[j].aic(dogcat)
    fig    = plot_wireframe(X, Y, Z, title=f'AIC = {AIC[j]:.4}' )
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0), scene_camera=camera ) # TODO: title???
    figs.append(pn.pane.Plotly(fig, width=250, height=300))

pn.Column( pn.Row(figs[0], figs[1]), pn.Row(figs[2],figs[3]))